# Installing Google Chrome

In [1]:
!wget https://dl.google.com/linux/linux_signing_key.pub
!sudo apt-key add linux_signing_key.pub
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' >> /etc/apt/sources.list.d/google-chrome.list
!apt-get update -y  
!sudo apt-get install -y google-chrome-stable
!wget -O /tmp/chromedriver.zip http://chromedriver.storage.googleapis.com/`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`/chromedriver_linux64.zip
!unzip /tmp/chromedriver.zip chromedriver -d /usr/local/bin/
!google-chrome --version
!chromedriver -v
!pip install selenium

from bs4 import BeautifulSoup
from PIL import Image
from selenium import webdriver
import requests
import os
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

--2023-07-24 04:43:48--  https://dl.google.com/linux/linux_signing_key.pub
Resolving dl.google.com (dl.google.com)... 74.125.135.190, 74.125.135.136, 74.125.135.91, ...
Connecting to dl.google.com (dl.google.com)|74.125.135.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14605 (14K) [application/octet-stream]
Saving to: ‘linux_signing_key.pub’

linux_signing_key.p 100%[===================>]  14.26K  --.-KB/s    in 0s      

2023-07-24 04:43:48 (40.9 MB/s) - ‘linux_signing_key.pub’ saved [14605/14605]

OK
Get:1 http://dl.google.com/linux/chrome/deb stable InRelease [1825 B]
Get:2 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5002 B]    
Get:3 https://packages.cloud.google.com/apt cloud-sdk InRelease [6361 B]       
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:6 http://dl.google.com/linux/chrome/deb stable/main amd64 Packages

# Start Scraping


In [ ]:
df = pd.read_csv('/kaggle/input/phonedataset/data_updated_v3.csv', sep=";")
prices_df = pd.DataFrame()
years_list = []
col_names = []
for j in range (len(df)):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("--window-size=1920,1080")
    
    driver = webdriver.Chrome(options=chrome_options)
    query = df.loc[j, "updated_names"].replace(" ","%20")
    
    col_names.append(df.loc[j, "updated_names"])
    print(query)
    #https://www.pricebook.co.id/search?keyword=Iphone+11
    url = 'https://www.pricebook.co.id/search?keyword='+query+'&product=all'
#     url = 'https://www.pricebook.co.id/OPPO-A12/40/PA_00001466'
#     url = 'https://www.pricebook.co.id/search?keyword=Samsung+Galaxy+A30&product=all'
    driver.get(url)
    time.sleep(2)

    try:
        product = WebDriverWait(driver, 5).until(
        (EC.element_to_be_clickable((By.CLASS_NAME, 'sc-bsipwG'))))
        product.click()
    except Exception as e:
        print()
    
#     try:
#         select_all = WebDriverWait(driver, 20).until(
#         (EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, "Semua"))))
#         select_all.click()
        
#     except Exception as e:
#         print(e)
    
    status = True
    time.sleep(2)
    while status == True:
        try:
            button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="Harga"]/div/div[2]/div[2]/div[2]/div[2]/div[2]/div[3]/button')))
            
            button.click()
        except Exception as e:
            status = False
            print()
            
    status = True

    while status == True:
        try:
            button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "pb-more-price")))
            button.click()
        except Exception as e:
            status = False
            print()
            
    status = True
    
    while status == True:
        try:
            button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "pb-shop-expand")))
            button.click()
        except Exception as e:
            status = False
            print()            

#     driver.save_screenshot('result.png')
#     img = Image.open('result.png')
#     img
    page_source = driver.page_source
    soup_source = BeautifulSoup(page_source, 'lxml')
    
    prices = []
    
    prices_selector_id = soup_source.find_all(id='pb-shop-price')
    #print(prices_selector_id)
    for i in range (len(prices_selector_id)):
        price = prices_selector_id[i].get_text().strip().replace("Rp ", "").replace(".","")
        prices.append(price)
        
    if (len(prices_selector_id) == 0):
        placeholder = [1]
        prices = pd.DataFrame(prices)
        placeholder = pd.DataFrame(placeholder)
        prices = pd.concat([placeholder, prices], axis=1, ignore_index=True)
        prices_df = pd.concat([prices_df, prices], axis=1, ignore_index=True)
        
    
    else:
        prices = pd.DataFrame(prices)
        prices_df = pd.concat([prices_df, prices], axis=1, ignore_index=True)
        
    
    
    prices_df.columns = col_names
    print(prices)
    
    driver.quit()
    
# df['release_years'] = years_list
# df2 = df
# df2.to_csv('data_years_added.csv', index=False)
prices_df.to_csv('datana_2.csv', index=False)